In [1]:
# -*- coding: utf-8 -*-
"""Resume Experience.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1av1PrKxhXUTah53mi2_6UephIk6BANvd
"""

'Resume Experience.ipynb\n\nAutomatically generated by Colab.\n\nOriginal file is located at\n    https://colab.research.google.com/drive/1av1PrKxhXUTah53mi2_6UephIk6BANvd\n'

In [2]:
# Install necessary packages
#!pip install streamlit langchain openai langchain_community chromadb PyPDF2 python-docx pyngrok pytest docx2txt

^C


  Using cached pypdf2-3.0.1-py3-none-any.whl.metadata (6.8 kB)
  Using cached python_docx-1.1.2-py3-none-any.whl.metadata (2.0 kB)
  Using cached altair-5.3.0-py3-none-any.whl.metadata (9.2 kB)
  Using cached blinker-1.8.2-py3-none-any.whl.metadata (1.6 kB)
  Using cached cachetools-5.3.3-py3-none-any.whl.metadata (5.3 kB)
  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
  Using cached pillow-10.3.0-cp311-cp311-win_amd64.whl.metadata (9.4 kB)
  Using cached pyarrow-16.1.0-cp311-cp311-win_amd64.whl.metadata (3.1 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached rich-13.7.1-py3-none-any.whl.metadata (18 kB)
  Using cached toml-0.10.2-py2.py3-none-any.whl.metadata (7.1 kB)
  Using cached GitPython-3.1.43-py3-none-any.whl.metadata (13 kB)
  Using cached pydeck-0.9.1-py2.py3-none-any.whl.metadata (4.1 kB)
  Using cached watchdog-4.0.1-py3-none-win_amd64.whl.metadata (37 kB)
  Using cached PyYAML-6.0.1-cp311-cp311-win_amd64.whl.metadata (2.1 kB

In [1]:
import os
import streamlit.components.v1 as components
import streamlit as st
from langchain.chains import RetrievalQA
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.callbacks.manager import CallbackManager
from langchain_community.llms import Ollama
from langchain_community.embeddings.ollama import OllamaEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory

In [2]:
# Simple passthrough function
def passthrough(input_data):
    return input_data

In [3]:
# Function to process each file and return a vectorstore
def process_file(file, embeddings):
    try:
        loader = PyPDFLoader(file)
        data = loader.load()
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=7500, chunk_overlap=100)
        chunks = text_splitter.split_documents(data)
        vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings)
        return vectorstore
    except Exception as e:
        st.error(f"Error processing file {file.name}: {e}")
        st.error(traceback.format_exc())
        return None

In [4]:
# Function to analyze inputs and return relevant information
def analyze_inputs(uploaded_files, job_ad_text, resume_text):
    try:
        if uploaded_files:
            embeddings = MistralAIEmbeddings()  # Use MistralAIEmbeddings without API key
            job_ad_vectorstore = None
            resume_vectorstore = None

            for file in uploaded_files:
                if "Job" in file.name:
                    job_ad_vectorstore = process_file(file, embeddings)
                else:
                    resume_vectorstore = process_file(file, embeddings)

            if job_ad_vectorstore and resume_vectorstore:
                retriever = MultiQueryRetriever.from_llm(
                    vector_db=job_ad_vectorstore.as_retriever(),
                    llm=ChatOllama(),
                    prompt=QUERY_PROMPT,
                )
                chain = (
                    {"context": retriever, "question": passthrough}
                    | PROMPT
                    | ChatOllama()
                    | passthrough  # Simple output parsing
                )
                response = chain.invoke("Analyze the resume based on the job description")
                return response

        return "Failed to process the files."
    except Exception as e:
        st.error(f"Error analyzing inputs: {e}")
        st.error(traceback.format_exc())
        return "Failed to analyze inputs."

In [5]:

# Streamlit UI components
def main():
    st.title("Intelligent Resume Matcher: Analyze Relevant Experience")

    st.write("This app will compare a job description to a resume and extract the number of years of relevant work experience from the resume.")

    uploaded_files = st.file_uploader("Upload Job Descriptions and Resumes", type=["pdf"], accept_multiple_files=True)
    job_ad_text = st.text_area("Job Description")
    resume_text = st.text_area("Resume Text")

    if st.button("Analyze"):
        with st.spinner("Processing..."):
            result = analyze_inputs(uploaded_files, job_ad_text, resume_text)
            st.write(result)

In [6]:
# Define the query prompt
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate three different versions of the given user question to retrieve relevant documents from a vector database. By generating multiple perspectives on the user question, your goal is to help the user overcome some of the limitations of the distance-based similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}""",
)

In [8]:
from langchain.prompts import ChatPromptTemplate
# Define the RAG prompt
PROMPT = ChatPromptTemplate.from_template(
    template="""Answer the question based ONLY on the following context:
    {context}
    Question: {question}"""
)

if __name__ == "__main__":
    main()

2024-06-28 16:28:37.762 
  command:

    streamlit run C:\Users\teres\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\ipykernel_launcher.py [ARGUMENTS]
2024-06-28 16:28:37.764 Session state does not function when running a script without `streamlit run`
